# NYC Yellow Taxi Tips Prediction With Machine Learning in Python

This example shows use of regression models to predict taxi tip fractions using the HPC-like platform Bodo. January - June 2019 NYC yellow taxi data is extracted from Bodo's public S3 bucket, cleaned and processed. Then some analysis are done to extract insight. All are **parallelized across multiple cores using Bodo**. This can be a straightforward way to make Python code run faster without a lot of changes to the code. 
Original example can be found [here](https://github.com/frenchlam/dask_CDSW/blob/master/03_Dask_ML-LargeDS.ipynb).

The current results are based on running on 2 `c5.2xlarge` instances (8 cores, 32GiB memory)

We downloaded first 6 months from 2019 `s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv)` and saved in parquet format in S3 Bucket (`s3://bodo-example-data/nyc-taxi/yellow_tripdata_01_06_2019.pq/`)

This data size was reduced to fit this hosted trial cluster.The full example using CSV dataset can be found in [Bodo-Examples Git repository](https://github.com/Bodo-inc/Bodo-examples/blob/master/ml/taxi-tips-prediction.ipynb)
You can run the large-scale example on [Bodo platform](https://platform.bodo.ai/account/login).


The Bodo framework knows when to parallelize code based on the `%%px` at the start of cells and `@bodo.jit` function decorators. Removing those and restarting the kernel will run the code without Bodo.

## Importing the Packages

These are the main packages we are going to work with:
 - Bodo to parallelize Python code automatically
 - Pandas to work with data
 - scikit-learn to build and evaluate regression models
 - xgboost for xgboost regressor model

In [1]:
%%px

import bodo
import pandas as pd
import numpy as np
import time

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
print(f"Hello World from rank {bodo.get_rank()}. Total ranks={bodo.get_size()}")

Starting 8 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/8 [00:00<?, ?engine/s]

%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

[stdout:3] Hello World from rank 3. Total ranks=8


[stdout:7] Hello World from rank 7. Total ranks=8


...

[stdout:1] Hello World from rank 1. Total ranks=8


[stdout:6] Hello World from rank 6. Total ranks=8


[stdout:4] Hello World from rank 4. Total ranks=8


[stdout:0] Hello World from rank 0. Total ranks=8


[stdout:2] Hello World from rank 2. Total ranks=8


## Load data

In [2]:
%%px

@bodo.jit(distributed=["taxi"], cache=True)
def get_taxi_trips():
    start = time.time()
    taxi = pd.read_parquet(
        "s3://bodo-example-data/nyc-taxi/yellow_tripdata_01_06_2019.pq/"
    )
    print("Reading time: ", time.time() - start)
    return taxi
    
taxi = get_taxi_trips()
if bodo.get_rank() == 0:
    display(taxi.shape)
    display(taxi.head())

%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

[stdout:0] Reading time:  7.077784639755237


[output:0]

(5557422, 18)

[output:0]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN


## Exploratory analysis

## Feature engineering

1. Create features before performing any data splitting.
2. Split data into train/test sets.

In [4]:
%%px

@bodo.jit(distributed=['taxi_df', 'df'], cache=True)
def prep_df(taxi_df):
    '''
    Generate features from a raw taxi dataframe.
    '''
    start = time.time()    
    df = taxi_df[taxi_df.fare_amount > 0]['tpep_pickup_datetime', 'passenger_count', 'tip_amount', 'fare_amount'].copy()  # avoid divide-by-zero
    df['tip_fraction'] = df.tip_amount / df.fare_amount
     
    df['pickup_weekday'] = df.tpep_pickup_datetime.dt.weekday
    df['pickup_weekofyear'] = df.tpep_pickup_datetime.dt.weekofyear
    df['pickup_hour'] = df.tpep_pickup_datetime.dt.hour
    df['pickup_week_hour'] = (df.pickup_weekday * 24) + df.pickup_hour
    df['pickup_minute'] = df.tpep_pickup_datetime.dt.minute
    df = df['pickup_weekday', 
    'pickup_weekofyear', 
    'pickup_hour', 
    'pickup_week_hour', 
    'pickup_minute', 
    'passenger_count', 'tip_fraction'].astype(float).fillna(-1)
    print("Data preparation time: ", time.time() - start)
    return df

taxi_feat = prep_df(taxi)
if bodo.get_rank() == 0:
    display(taxi_feat.head())

%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

[stdout:0] Data preparation time:  2.578672713628407


[output:0]

,pickup_weekday,pickup_weekofyear,pickup_hour,pickup_week_hour,pickup_minute,passenger_count,tip_fraction
0,1.0,1.0,0.0,24.0,46.0,1.0,0.235714
1,1.0,1.0,0.0,24.0,59.0,1.0,0.071429
2,4.0,51.0,13.0,109.0,48.0,3.0,0.000000
3,2.0,48.0,15.0,63.0,52.0,5.0,0.000000
4,2.0,48.0,15.0,63.0,56.0,5.0,0.000000


In [5]:
%%px

@bodo.jit(distributed=["taxi_feat", "X_train", "X_test", "y_train", "y_test"])
def data_split(taxi_feat):
    X_train, X_test, y_train, y_test = train_test_split(
        taxi_feat['pickup_weekday', 
                    'pickup_weekofyear', 
                    'pickup_hour', 
                    'pickup_week_hour', 
                    'pickup_minute', 
                    'passenger_count'], 
        taxi_feat['tip_fraction'], 
        test_size=0.3,
        train_size=0.7,        
        random_state=42
    )
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = data_split(taxi_feat)

%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

## Train Model over large dataset

We'll train a linear model to predict tip_fraction and evaluate these models against the test set using RMSE.

#### 1. Linear Regression

In [7]:
%%px

@bodo.jit(distributed=['X_train', 'y_train', 'X_test', 'y_test'], cache=True)
def lr_model(X_train, y_train, X_test, y_test):
    start = time.time()    
    lr = LinearRegression()
    lr_fitted = lr.fit(X_train.to_numpy(), y_train.values)
    print("Linear Regression fitting time: ", time.time() - start)

    start = time.time()    
    lr_preds = lr_fitted.predict(X_test.to_numpy())
    print("Linear Regression prediction time: ", time.time() - start)    
    print(mean_squared_error(y_test, lr_preds, squared=False))
    
lr_model(X_train, y_train, X_test, y_test)

%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

[stderr:0] /tmp/ipykernel_23511/2291481960.py:13: BodoWarning: Data is distributed so Bodo will fit model with SGD solver optimization (SGDRegressor)
  lr_model(X_train, y_train, X_test, y_test)


[stdout:0] Linear Regression fitting time:  18.852877387157378
Linear Regression prediction time:  0.09847916721514594
18.01958065782363


#### 2. Ridge

In [8]:
%%px

@bodo.jit(distributed=['X_train', 'y_train', 'X_test', 'y_test'])
def rr_model(X_train, y_train, X_test, y_test):
    start = time.time()    
    rr = Ridge()
    rr_fitted = rr.fit(X_train.to_numpy(), y_train.values)
    print("Ridge fitting time: ", time.time() - start)

    start = time.time()    
    rr_preds = rr_fitted.predict(X_test.to_numpy())
    print("Ridge prediction time: ", time.time() - start)    
    print(mean_squared_error(y_test, rr_preds, squared=False))
    
rr_model(X_train, y_train, X_test, y_test)

%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

[stderr:0] /tmp/ipykernel_23511/3582817498.py:13: BodoWarning: Data is distributed so Bodo will fit model with SGD solver optimization (SGDRegressor)
  rr_model(X_train, y_train, X_test, y_test)


[stdout:0] Ridge fitting time:  17.40331115617414
Ridge prediction time:  0.10323073055758414
18.012137437147715


#### 3. Lasso

In [9]:
%%px

@bodo.jit(distributed=['X_train', 'y_train', 'X_test', 'y_test'])
def lsr_model(X_train, y_train, X_test, y_test):
    start = time.time()    
    lsr = Lasso()
    lsr_fitted = lsr.fit(X_train.to_numpy(), y_train.values)
    print("Lasso fitting time: ", time.time() - start)

    start = time.time()    
    lsr_preds = lsr_fitted.predict(X_test.to_numpy())
    print("Lasso prediction time: ", time.time() - start)    
    print(mean_squared_error(y_test, lsr_preds, squared=False))
    
lsr_model(X_train, y_train, X_test, y_test)

[stderr:0] /tmp/ipykernel_23511/3478443939.py:13: BodoWarning: Data is distributed so Bodo will fit model with SGD solver optimization (SGDRegressor)
  lsr_model(X_train, y_train, X_test, y_test)


%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

[stdout:0] Lasso fitting time:  18.898461740320272
Lasso prediction time:  0.0993677500304102
18.012099237486996


#### 4. SGDRegressor

In [10]:
%%px

@bodo.jit(distributed=['X_train', 'y_train', 'X_test', 'y_test'])
def sgdr_model(X_train, y_train, X_test, y_test):
    start = time.time()    
    sgdr = SGDRegressor(max_iter=100, penalty='l2')
    sgdr_fitted = sgdr.fit(X_train.to_numpy(), y_train.values)
    print("SGDRegressor fitting time: ", time.time() - start)

    start = time.time()    
    sgdr_preds = sgdr_fitted.predict(X_test.to_numpy())
    print("SGDRegressor prediction time: ", time.time() - start)    
    print(mean_squared_error(y_test, sgdr_preds, squared=False))
    
sgdr_model(X_train, y_train, X_test, y_test)

%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

[stdout:0] SGDRegressor fitting time:  19.750688465580424
SGDRegressor prediction time:  0.1005445906064324
18.012842013830316


#### 5. XGBoost Model

In [11]:
%%px
@bodo.jit(distributed=['X_train', 'y_train', 'X_test', 'y_test'], cache=True)
def xgb_model(X_train, y_train, X_test, y_test):
    start = time.time()    
    xgb = XGBRegressor(
        objective="reg:squarederror",
        tree_method='approx',
        learning_rate=0.1,
        max_depth=5,
        n_estimators=100,
    )
    xgb_fitted = xgb.fit(X_train, y_train)
    print("XGBRegressor fitting time: ", time.time() - start)

    start = time.time()    
    xgb_preds = xgb_fitted.predict(X_test)
    print("XGBRegressor prediction time: ", time.time() - start)    
    print(mean_squared_error(y_test, xgb_preds, squared=False))
    
xgb_model(X_train, y_train, X_test, y_test)    

%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

[stdout:0] XGBRegressor fitting time:  540.2045168466317
XGBRegressor prediction time:  2.4025714681524732
18.064790054522856
